## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-11-06-27-53 +0000,wapo,ICE accused of pepper-spraying 1-year-old girl...,https://www.washingtonpost.com/immigration/202...
1,2025-11-11-06-08-27 +0000,nypost,Florida 15-year-old charged with felony after ...,https://nypost.com/2025/11/11/us-news/florida-...
2,2025-11-11-05-34-13 +0000,nypost,Former Jan. 6 defendant who shot gun in air du...,https://nypost.com/2025/11/11/us-news/former-j...
3,2025-11-11-05-32-33 +0000,nyt,The BBC Under Fire,https://www.nytimes.com/2025/11/10/world/bbc-t...
4,2025-11-11-05-18-02 +0000,nyt,New Rules Aim to Broaden Appeal of H.S.A.s,https://www.nytimes.com/2025/11/07/your-money/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2341/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
98,trump,41
53,shutdown,33
119,government,22
35,new,17
52,end,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
66,2025-11-11-01-13-00 +0000,wsj,Trump threatened to dock the pay of air-traffi...,https://www.wsj.com/politics/policy/trump-thre...,140
51,2025-11-11-02-00-00 +0000,wsj,Senate Minority Leader Chuck Schumer wasn’t on...,https://www.wsj.com/politics/policy/progressiv...,131
148,2025-11-10-21-11-40 +0000,nypost,Trump pledges to ‘abide’ by deal to end govern...,https://nypost.com/2025/11/10/us-news/trump-pl...,128
33,2025-11-11-03-12-00 +0000,wsj,The decision by eight members of the Senate De...,https://www.wsj.com/politics/policy/democrats-...,127
36,2025-11-11-02-55-00 +0000,wsj,The Republican-led Senate late Monday passed a...,https://www.wsj.com/politics/policy/senate-pas...,118


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
66,140,2025-11-11-01-13-00 +0000,wsj,Trump threatened to dock the pay of air-traffi...,https://www.wsj.com/politics/policy/trump-thre...
51,76,2025-11-11-02-00-00 +0000,wsj,Senate Minority Leader Chuck Schumer wasn’t on...,https://www.wsj.com/politics/policy/progressiv...
256,67,2025-11-10-14-58-51 +0000,nypost,Supreme Court rejects Kim Davis bid to overtur...,https://nypost.com/2025/11/10/us-news/supreme-...
254,56,2025-11-10-15-03-47 +0000,cbc,8 people dead in car explosion near Red Fort i...,https://www.cbc.ca/news/world/india-explosion-...
267,47,2025-11-10-14-01-26 +0000,nypost,Ex-French President Nicolas Sarkozy to be rele...,https://nypost.com/2025/11/10/world-news/franc...
233,43,2025-11-10-16-35-00 +0000,wsj,Sen. Bernie Sanders and a group of Democratic ...,https://www.wsj.com/politics/policy/the-politi...
82,42,2025-11-11-00-04-13 +0000,nypost,"Major break in Long Island cold case murder, r...",https://nypost.com/2025/11/10/us-news/major-br...
49,35,2025-11-11-02-12-03 +0000,nyt,Trump Threatens to Sue the BBC for $1 Billion ...,https://www.nytimes.com/2025/11/10/world/europ...
221,34,2025-11-10-16-52-16 +0000,nypost,"Typhoon Fung-wong leaves 8 dead, 1.4 million d...",https://nypost.com/2025/11/10/world-news/8-dea...
279,32,2025-11-10-12-30-00 +0000,wsj,Private-Label Food Maker TreeHouse to Go Priva...,https://www.wsj.com/business/retail/private-la...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
